# Pokémon competition

In this notebook you have to provide the best pipeline that you have found to predict Pokémon battles.

At the end you will have to generate a set of predictions over the unlabeled data `data.hidden` and `data_inverse.hidden`. In these unlabeled dataset you will find all the Pokémon battles that we will be performing in some *fictional* Pokémon competition, so we do not know the outcome of these battles right now!

Remember to use all the tools that we have seen in class to evaluate and fine-tune your pipeline.

*Gotta Predict 'Em All!*

Paste here your pipeline:

In [3]:
# Data: Replace it with the data from the provided datasets
from sklearn.model_selection import train_test_split
from pathlib import Path
import pandas
import joblib

__wd__ = Path("__file__").resolve().parent

datasets_path = __wd__ / "datasets"

data = pandas.read_csv(datasets_path / "data.train", index_col=0)
inverse_data = pandas.read_csv(datasets_path / "data_inverse.train", index_col=0)

def get_Xy(dataset):
    return dataset.drop("Wins", axis=1), dataset["Wins"]

X, y = get_Xy(data)

columns_to_drop = ['Legendary', 'Legendary__other', 'Name', 'Name__other','Generation','Generation__other']
used_features = X.drop(columns=columns_to_drop, axis=1)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(used_features, y, test_size=0.2, random_state=42)

pipeline = joblib.load('pipeline.pkl')

pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['HP', 'Attack', 'Defense',
                                                   'Sp. Atk', 'Sp. Def',
                                                   'Speed', 'HP__other',
                                                   'Attack__other',
                                                   'Defense__other',
                                                   'Sp. Atk__other',
                                                   'Sp. Def__other',
                                                   'Speed__other']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Type 1',
                                                   'Type 1__other'])])),
                ('classifier',
                 VotingClassifier(estimators=[('decision_tree',
                                               DecisionTreeClassifier(max_depth=7,
                                                                      random_state=42)),
                                              ('random_forest',
                                               RandomForestClassifier(n_estimators=150,
                                                                      random_state=42)),
                                              ('mlp_classifier',
                                               MLPClassifier(hidden_layer_sizes=(150,),
                                                             random_state=42))]))])

In [2]:
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!
import pandas
from pathlib import Path

__wd__ = Path("__file__").resolve().parent
datasets_path = __wd__ / "datasets"

tournament = pandas.read_csv(datasets_path / "data.hidden")
tournament_inverse = pandas.read_csv(datasets_path / "data_inverse.hidden")

y_predicted = pipeline.predict(tournament)
y_inverse_predicted = pipeline.predict(tournament_inverse)

y_predicted.tofile("predicted.csv", sep=",")
y_inverse_predicted.tofile("predicted_inverse.csv", sep=",")
# !!!!!!!!!!!!!!!!!!!!!!!!
# Do not change this code
# !!!!!!!!!!!!!!!!!!!!!!!!